Setup and Load Model Forecasts

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# Load model forecasts
# Each should be: date × issue_type_sg columns with predicted counts
df_forecast_lstm = pd.read_csv("./outputs/forecast_lstm.csv", parse_dates=["date"])
df_forecast_tft = pd.read_csv("./outputs/forecast_tft.csv", parse_dates=["date"])
df_forecast_mavg = pd.read_csv("./outputs/forecast_moving_avg.csv", parse_dates=["date"])  # optional

# Load latest historical actuals
df_actual = pd.read_csv("./data/processed/timeseries_issues_by_type.csv", parse_dates=["date"])


Plot Forecast vs Actual (Selected Categories)

In [ ]:
issue_types = ["Cleanliness", "Roads & Footprints", "Animals & Bird"]

for issue in issue_types:
    plt.figure(figsize=(10, 4))

    # Historical actuals
    df_plot_actual = df_actual[df_actual["issue_type_sg"] == issue].set_index("date")["issue_count"]
    df_plot_actual.tail(60).plot(label="Actual", color="black")

    # Forecasts
    df_forecast_lstm.set_index("date")[issue].plot(label="LSTM", linestyle="--")
    df_forecast_tft.set_index("date")[issue].plot(label="TFT", linestyle=":")
    df_forecast_mavg.set_index("date")[issue].plot(label="Moving Avg", linestyle="dashdot")

    plt.axvline(df_plot_actual.index.max(), color="red", linestyle="--", label="Forecast Start")
    plt.title(f"Issue Count Forecast for: {issue}")
    plt.ylabel("Predicted Count")
    plt.legend()
    plt.tight_layout()
    plt.show()


Combine and Export Forecasts (Wide Format)

In [ ]:
# Add model source labels
df_forecast_lstm["model"] = "LSTM"
df_forecast_tft["model"] = "TFT"
df_forecast_mavg["model"] = "Moving Avg"

# Combine all
df_forecast_all = pd.concat([df_forecast_lstm, df_forecast_tft, df_forecast_mavg], axis=0)

# Melt to long format for dashboard
df_long = df_forecast_all.melt(id_vars=["date", "model"], var_name="issue_type_sg", value_name="predicted_count")

# Export for dashboard integration
df_long.to_csv("./outputs/forecast_combined_long.csv", index=False)


Summary Plot – Total Forecasted Issue Count

In [ ]:
# Sum across all issue types per model
df_long["date"] = pd.to_datetime(df_long["date"])
summary_df = df_long.groupby(["model", "date"])["predicted_count"].sum().reset_index()

plt.figure(figsize=(10, 5))
sns.lineplot(data=summary_df, x="date", y="predicted_count", hue="model")
plt.title("Total Forecasted Issue Count (All Categories)")
plt.ylabel("Total Issues")
plt.xlabel("Date")
plt.tight_layout()
plt.show()


Summary

### Forecasting Results Summary

- Forecasts generated using LSTM, TFT, and Moving Average for next 14 days.
- Issue types include Cleanliness, Roads & Footprints, Animals & Bird, etc.
- Visualizations compare actual vs forecasted counts.
- Exported long-format forecast file for dashboard or web integration: `forecast_combined_long.csv`
